In [128]:
!pip install sqlite_object
!pip install sqlitedict

In [131]:
import numpy as np
import scipy.stats
from sqlitedict import SqliteDict
from sqlite_object import SqliteList

#Список уникальных клиентов
unique_clients = SqliteList(filename='./unique_clients',  commit_every=10000)

amount_of_ppl_in_sgmnt = SqliteDict('./amount_of_ppl_in_sgmnt', autocommit=True) 
#количество уникальных клиентов для каждого из сегментов

values_in_sgmnt = SqliteDict('./values_in_sgmnt', autocommit=True) 
#транзакции для каждого сегмента

with open(r"C:\Users\anton\Desktop\Учеба\raiffeisen\transactions.txt") as file_handler:
    for line in file_handler:
        line = line.replace("\n", "")
        splitted_line = line.split(',')
        client = splitted_line[1]
        segment = splitted_line[3]
        if segment not in amount_of_ppl_in_sgmnt:
            amount_of_ppl_in_sgmnt[segment] = 0
        if client not in unique_clients:
            amount_of_ppl_in_sgmnt[segment] += 1
            unique_clients.append(client)
            
        if segment not in values_in_sgmnt:
            values_in_sgmnt[segment] = []
        values_in_sgmnt[segment] += [int(splitted_line[2])]

print("Задание 1 \nКоличество клиентов для каждого из сегментов")
for key, val in amount_of_ppl_in_sgmnt.items():
    print(key, ':', val)
print()

ans = {}
for sgmnt in values_in_sgmnt:
    a = 1.0 * np.array(values_in_sgmnt[sgmnt])
    n = a.shape[0]
    mean = np.mean(a)
    stand_err = scipy.stats.sem(a)
    interval = scipy.stats.t.interval(0.9, n-1, loc=mean, scale=stand_err)
    ans[sgmnt] = (mean, interval)
    
print("Задание 2 \nСредний объем отдельной транзакции в каждом из сегментов.")
for key, val in ans.items():
    print(key, ":", val[0])
print()

print("Задание 3 \n90% доверительный интервал для среднего объема отдельной транзакции в каждом из сегментов.")
for key, val in ans.items():
    print(key, ":", val[1])
print()

print("Задание 4")
p_value = scipy.stats.ttest_ind(values_in_sgmnt['R'], values_in_sgmnt['AF'])[1]
if p_value > 0.1:
    print("""Т. к. {} > 0.1, то гипотеза о равенстве средних объемов отдельных транзакций между сегментами 
может быть принята при уровне значимости 10%""".format(p_value))
else:
    print("""Т. к. {} < 0.1, то гипотеза о равенстве средних объемов отдельных транзакций между сегментами 
не может быть принята при уровне значимости 10%""".format(p_value))

    
amount_of_ppl_in_sgmnt.clear()
values_in_sgmnt.clear()
unique_clients.clear()

Задание 1 
Количество клиентов для каждого из сегментов
R : 2
AF : 2

Задание 2 
Средний объем отдельной транзакции в каждом из сегментов.
R : 2486.6666666666665
AF : 35300.0

Задание 3 
90% доверительный интервал для среднего объема отдельной транзакции в каждом из сегментов.
R : (-42.4147143068285, 5015.74804764016)
AF : (-183787.1775635926, 254387.17756359235)

Задание 4
Т. к. 0.2944172804761428 > 0.1, то гипотеза о равенстве средних объемов отдельных транзакций между сегментами 
может быть принята при уровне значимости 10%
